협업 필터링 기반 추천시스템

In [1]:
# KNN(최근접 이웃 알고리즘)
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

In [2]:
# 사용자 -아이템 메트릭스(사용자가 평가한 영화 점수)
data = {
    'movie1' : [5,4,1,0],
    'movie2' : [4,0,2,1],
    'movie3' : [1,1,0,5],
    'movie4' : [0,2,4,4],
}

In [6]:
rating = pd.DataFrame(data, index=['user1', 'user2', 'user3', 'user4'])
rating

,movie1,movie2,movie3,movie4
user1,5,4,1,0
user2,4,0,1,2
user3,1,2,0,4
user4,0,1,5,4


In [8]:
# knn 모델
# brute : 전수조사 방식 (Brute Force), 데이터셋의 모든 점들과 하나씩 거리 계산을 하고 가장 가까운 이웃을 찾는 방식 : 데이터가 작고 정교할때, 데이터가 크면 비효율적
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(rating)

NearestNeighbors(algorithm='brute', metric='cosine')

In [22]:
distance,indexs =  knn.kneighbors(rating.loc['user1'].values.reshape(1,-1),n_neighbors=2 )

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [30]:
print(f"distance : {distance},  indexs : {indexs}, {np.array(rating.index)[[indexs]]}")

distance : [[0.         0.29289322]],  indexs : [[0 1]], [[['user1' 'user2']]]


In [29]:
np.array(rating.index)[[0,1]]

array(['user1', 'user2'], dtype=object)

확률적 경사하강법
   - 행렬 분해를 통해 사용자와 아이템을 잠재요인으로 분해 이를통해 사용자-아이템 상호 작용을 예측
   - 매우큰 데이터셋에서도 사용가능 빠르게 학습할수 있다

In [31]:
# 행렬 분해
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
# 사용자 - 아이템 매트릭스
rating_matrix = rating.values
rating_matrix

array([[5, 4, 1, 0],
       [4, 0, 1, 2],
       [1, 2, 0, 4],
       [0, 1, 5, 4]])

In [33]:
# 훈련 / 테스트 데이터 분할
train_matrix, test_matrix = train_test_split(rating_matrix, test_size=0.2, random_state=42)
train_matrix, test_matrix

(array([[0, 1, 5, 4],
        [5, 4, 1, 0],
        [1, 2, 0, 4]]),
 array([[4, 0, 1, 2]]))

In [35]:
# SGD를 이용한 행렬 분해
svd = TruncatedSVD(n_components=2,random_state=42)  #  n_components=2 분해된 차원의 수 , 원본행렬에서 2개의 주요 잠재요인을 추출
train_svd = svd.fit_transform(train_matrix)
train_svd, train_matrix

(array([[ 5.2470859 , -3.59214204],
        [ 4.62155673,  4.48902164],
        [ 3.70983358, -0.51161064]]),
 array([[0, 1, 5, 4],
        [5, 4, 1, 0],
        [1, 2, 0, 4]]))

In [36]:
svd.components_

array([[ 0.42803021,  0.497226  ,  0.4925017 ,  0.57183784],
       [ 0.65833649,  0.40042337, -0.40435431, -0.49269847]])

In [37]:
# 예측결과
# 축소된 사용자 잠재요인 행렬과 아이템 잠재요인 행렬읠 곱 을 하면 원본평점행렬의 근사치를 계산
predicted_ratings =  np.dot(train_svd,svd.components_)
predicted_ratings


array([[-0.11892693,  1.1706099 ,  4.03669686,  4.77032516],
       [ 4.93345266,  4.09546732,  0.46096933,  0.43104696],
       [ 1.25110888,  1.63976485,  2.03397133,  2.37349301]])

In [43]:
# 성능평가 mse
mse = mean_squared_error(train_matrix,predicted_ratings)
mse

0.7524910527742803